In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#!pip install -r requirment.txt

In [5]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [6]:
os.environ["GROQ_API_KEY"] = "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"

In [11]:
# Fetch the PDF from the URL
import requests
url = "https://dspmuranchi.ac.in/pdf/Blog/Python%20Built-In%20Functions.pdf"
response = requests.get(url)

In [12]:
# Save the PDF to a local file
with open("python_inbuildfunction.pdf", "wb") as f:
    f.write(response.content)

In [13]:
# laoding the document
loader = UnstructuredFileLoader("python_inbuildfunction.pdf")

In [14]:
documents = loader.load()
documents

[Document(metadata={'source': 'python_inbuildfunction.pdf'}, page_content='Python Built-In Functions\n\nGaurav Kr. suman\n\nMIT5\n\n1. abs()\n\nThe abs() is one of the most popular Python built -in functions, which returns the absolute value of a number. A negative value’s absolute is that value is positive.\n\n>>> abs(-7)\n\n7\n\n>>> abs(7)\n\n7\n\n>>> abs(0)\n\n2. all()\n\nThe all() function takes a container as an argument. This Built in Functions returns True if all values in a python iterable have a Boolean value of True. An empty value has a Boolean value of False.\n\n>>> all({\'*\',\'\',\'\'})\n\nFalse\n\n>>> all([\' \',\' \',\' \'])\n\nTrue\n\n3. any()\n\nLike all(), it takes one argument and returns True if, even one value in the iterable has a Boolean value of True.\n\n>>> any((1,0,0))\n\nTrue\n\n>>> any((0,0,0))\n\nFalse\n\n4. ascii()\n\nIt is important Python built-in functions, returns a printable representation of a python object (like a string or a Python list). Let’s ta

In [17]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)


In [18]:
texts = text_splitter.split_documents(documents)

In [19]:
type(texts)

list

In [20]:
len(texts)

7

In [21]:
texts[4]

Document(metadata={'source': 'python_inbuildfunction.pdf'}, page_content='frozenset() returns an immutable frozenset object.\n\n8 | P a g e\n\n>>> frozenset((3,2,4))\n\nfrozenset({2, 3, 4})\n\nRead Python Sets and Booleans for more on frozenset.\n\n24. getattr()\n\ngetattr() returns the value of an object’s attribute.\n\n>>> getattr(orange,\'size\')\n\n7\n\n25. globals()\n\nThis Python built-in functions, returns a dictionary of the current global symbol table.\n\n>>> globals()\n\n{‘__name__’: ‘__main__’, ‘__doc__’: None, ‘__package__’: None, ‘__loader__’: <class ‘_frozen_importlib.BuiltinImporter’>, ‘__spec__’: None, ‘__annotations__’: {}, ‘__builtins__’: <module ‘builtins’ (built-in)>, ‘fruit’: <class ‘__main__.fruit’>, ‘orange’: <__main__.fruit object at 0x05F937D0>, ‘a’: 2, ‘numbers’: [1, 2, 3], ‘i’: (2, 3), ‘x’: 7, ‘b’: 3}\n\n26. hasattr()\n\nLike delattr() and getattr(), hasattr() Python built-in functions, returns True if the object has that attribute.\n\n>>> hasattr(orange,\'si

In [22]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-22-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
persist_directory = "vector_db"

In [24]:
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [25]:
# retriever
retriever = vectordb.as_retriever()

In [26]:
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

In [27]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [28]:
# invoke the qa chain and get a response for user query
query = "what are the function from this pdf "
response = qa_chain.invoke({"query": query})

In [29]:
print(response)

{'query': 'what are the function from this pdf ', 'result': 'The functions mentioned in the provided PDF are:\n\n1. abs()\n2. all()\n3. any()\n4. ascii()\n5. bin()\n6. bool()\n7. bytearray()\n8. bytes()\n9. divmod()\n10. enumerate()\n11. eval()\n12. exec()\n13. filter()\n14. float()\n15. format()\n16. frozenset()\n17. getattr()\n18. globals()\n19. input()\n20. int()\n21. isinstance()\n22. ord()\n23. pow()\n24. print()\n25. range()\n26. repr()\n27. reversed()\n28. round()\n29. set()\n30. setattr()\n\nThese are all built-in functions in Python.', 'source_documents': [Document(metadata={'source': 'python_inbuildfunction.pdf'}, page_content='>>> divmod(3,7)\n\n6 | P a g e\n\n(0, 3)\n\n>>> divmod(7,3)\n\n(2, 1) If you encounter any doubt in Python Built-in Function, Please Comment.\n\n17. enumerate()\n\nThis Python Built In function returns an enumerate object. In other words, it adds a counter to the iterable.\n\n>>> for i in enumerate([\'a\',\'b\',\'c\']):\n\nprint(i)\n\n(0, ‘a’) (1, ‘b’)

In [30]:
print(response["result"])

The functions mentioned in the provided PDF are:

1. abs()
2. all()
3. any()
4. ascii()
5. bin()
6. bool()
7. bytearray()
8. bytes()
9. divmod()
10. enumerate()
11. eval()
12. exec()
13. filter()
14. float()
15. format()
16. frozenset()
17. getattr()
18. globals()
19. input()
20. int()
21. isinstance()
22. ord()
23. pow()
24. print()
25. range()
26. repr()
27. reversed()
28. round()
29. set()
30. setattr()

These are all built-in functions in Python.


In [31]:
print(response["source_documents"][0].metadata["source"])

python_inbuildfunction.pdf


In [32]:
# invoke the qa chain and get a response for user query
query = "Give me summary of all function from this pdf?"
response = qa_chain.invoke({"query": query})
print(response["result"])
print("*"*30)
print("Source Document:", response["source_documents"][0].metadata["source"])

Here's a summary of the Python built-in functions mentioned in the PDF:

1. `abs()`: Returns the absolute value of a number.
2. `all()`: Returns `True` if all values in an iterable are `True`.
3. `any()`: Returns `True` if at least one value in an iterable is `True`.
4. `ascii()`: Returns a printable representation of a Python object.
5. `bin()`: Converts an integer to a binary string.
6. `bool()`: Converts a value to a boolean.
7. `bytearray()`: Returns a mutable array of bytes.
8. `bytes()`: Returns an immutable array of bytes.
9. `chr()`: Returns a string representing a character whose Unicode code point is the integer.
10. `divmod()`: Returns the quotient and remainder of two numbers.
11. `enumerate()`: Returns an iterator that produces tuples containing a count and a value from an iterable.
12. `eval()`: Evaluates a string as a Python expression.
13. `exec()`: Executes a string as Python code.
14. `filter()`: Returns an iterator that filters elements from an iterable based on a co